In [1]:
import sys; sys.path.insert(0, '..')
from gquant.dataframe_flow import TaskGraph

In [2]:
from dask_cuda import LocalCUDACluster
cluster = LocalCUDACluster()
from dask.distributed import Client
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:34563 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 100.00 GB


We can open the status page in the brwoser by following javascript commands

In [3]:
from IPython.display import HTML
javascript = """
<script type="text/Javascript">
    function check_status(){
        var url = document.location.href;
        var index = url.indexOf(':8888');
        var status = url.substr(0, index)+":8787";
        window.open(status,'_blank');
    }
    check_status();
</script>
"""
HTML(javascript)

# Prepare distributed CSV files <span style="color:red">(not necessary if the dataset is already prepared)</span>

Following is the code to prepare the dataset as mulitple csv files that we can use dask_cuda to load them

First use this simple taskgraph to load data then sort it by the asset id and datatime

In [2]:
task_graph = TaskGraph.load_taskgraph('../taskgraphs/sort_stocks.gq.yaml')
input_cached, = task_graph.run()
task_graph.draw()

GQuantWidget(sub=HBox(), value=[OrderedDict([('id', 'stock_data'), ('type', 'CsvStockLoader'), ('conf', {'file…

convert the sorted stock data into partitions and save it into csv files. Note, the data is slited in a way that the same asset belongs to the same partition

In [ ]:
import dask.dataframe as dd
import os
num_partitions = 8

os.makedirs('many-small', exist_ok=True)
dd.from_pandas(input_cached.set_index('asset'), npartitions=num_partitions).reset_index().to_csv('many-small/*.csv', index=False)

In [4]:
!cat ../taskgraphs/dask_tutorial.gq.yaml

- id: stock_data
  type: CsvStockLoader
  conf:
    file: /home/quant/gQuant/notebooks/data/stock_price_hist.csv.gz
    path: /home/quant/gQuant/notebooks/many-small
  inputs: {}
- id: sort_node
  type: SortNode
  conf:
    keys:
      - asset
      - datetime
  inputs:
    in: stock_data.dask_cudf_out
- id: ""
  type: Output_Collector
  conf: {}
  inputs:
    in1: output_csv.df_out
- id: average_volume
  type: AverageNode
  conf:
    column: volume
  inputs:
    stock_in: sort_node.out
- id: output_csv
  type: OutCsvNode
  conf:
    path: /home/quant/gQuant/notebooks/dask_average_volume.csv
  inputs:
    df_in: average_volume.stock_out


In [5]:
task_graph = TaskGraph.load_taskgraph('../taskgraphs/dask_tutorial.gq.yaml')
task_graph.draw()

GQuantWidget(sub=HBox(), value=[OrderedDict([('id', 'stock_data'), ('type', 'CsvStockLoader'), ('conf', {'file…

In [6]:
task_graph.run(formated=True, profile=True)

In [7]:
task_graph.run()[0].compute()

,asset,volume
0,93,84.985000
1,165,67.066667
2,239,128.715368
3,281,100.566667
4,592,103.425000
...,...,...
5012,869589,161.796051
5013,869590,203.910068
5014,869592,101.498676
5015,869597,81.415663


clean up the generated the csv file

In [8]:
!rm dask_average_volume.csv